# DataFrame examples for screencasts


This notebook contains the non-trivial examples used in the slides. In the long run 
we want to convert all DataFrame examples to tasks. 

The examples that are not in this notebook can be created trivially by using the 
`get_gapminder_sample` function with the correct argument and executing code from the 
slides. 

In [ ]:
import pandas as pd

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"

import plotly.express as px

from epp_topics.slidev_utilities import get_html


def get_gapminder_sample(version="tiny"):
    df = px.data.gapminder()
    df = df.rename(
        columns={
            "lifeExp": "life_exp",
            "gdpPercap": "gdp_per_cap",
        },
    )
    if version == "tiny":
        df = df[["country", "continent", "year", "life_exp"]]
        df = df.query("country in ['Cuba', 'Spain']")
        df = df.query("year in [2002, 2007]")
    elif version == "full":
        pass
    elif version == "all_observations":
        df = df[["country", "continent", "year", "life_exp"]]
    elif version == "income":
        df = df[["country", "year", "gdp_per_cap", "pop"]]
        df = df.query("country in ['Cuba', 'Spain']")
        df = df.query("year in [2002, 2007]")
    else:
        msg = f"Unknown version: {version}"
        raise ValueError(msg)

    return df.reset_index(drop=True)

## Example for inspecting and summarizing data

This recreates the look of the full DataFrame in a pandas notebook at default display
settings. 

In [ ]:
# gapminder full (pandas/inspecting_and_summarizing/screencast)

df = get_gapminder_sample("full")
df = pd.concat([df.loc[:6], df.loc[1699:]])
df.loc[6] = "..."
new_index = df.index.tolist()
new_index[6] = "..."
df.index = new_index
print(get_html(df))
df

## Gapminder data in wide format for the normal forms / rules for data management screencast

In [ ]:
# gapminder in wide format (pandas/rules/screencast)

df = get_gapminder_sample("income")

df_wide = df.pivot(index="country", columns="year", values=["gdp_per_cap", "pop"])

df_wide.columns = df_wide.columns.map(lambda x: f"{x[0]}_{x[1]}")

print(get_html(df_wide))
df_wide

In [ ]:
# gapminder in long format (pandas/rules/screencast)

df = get_gapminder_sample("income")
print(get_html(df))
df

## Combining dataframes

In [ ]:
# concat without axis argument
# (pandas/columns_and_indices/screencast  -- The dataframe from before)

df = get_gapminder_sample("tiny")
top = df.query("country == 'Cuba'")
bottom = df.query("country == 'Spain'")

df = pd.concat([top, bottom])
print(get_html(df))
df

In [ ]:
# concat without axis argument (Concatenating DataFrames vertically)
# (pandas/columns_and_indices/screencast  -- Same dataset, different Index)

df = get_gapminder_sample("tiny").set_index(["country", "year"])
top = df.query("country == 'Cuba'")
bottom = df.query("country == 'Spain'")

df = pd.concat([top, bottom])
print(get_html(df))
df

In [ ]:
# concat with axis = 1

# From: pandas merging


left = get_gapminder_sample("tiny").set_index(["country", "year"])
right = get_gapminder_sample("income").set_index(["country", "year"])

df = pd.concat([left, right], axis="columns")
print(get_html(df))
df

In [ ]:
# 1:1 merge (pandas/merging/screencast)

left = get_gapminder_sample("tiny").iloc[:3].reset_index(drop=True)
right = get_gapminder_sample("income").iloc[1:].reset_index(drop=True)

df = pd.merge(
    left=left,
    right=right,
    on=["country", "year"],
    how="outer",
)

print(get_html(df))
df

In [ ]:
# m:1 merge

left = get_gapminder_sample("tiny")[["country", "year", "life_exp"]]

right = pd.DataFrame()
right["country"] = ["Cuba", "Spain"]
right["capital"] = ["Havana", "Madrid"]

df = pd.merge(
    left=left,
    right=right,
    on="country",
)

print(get_html(df))
df

## Pandas: What is (modern) pandas?

In [ ]:
# What is a DataFrame?

df = get_gapminder_sample("tiny")

print(get_html(df))
df

## Pandas: Loading and Saving

In [ ]:
# Example: Loading a csv file

af = get_gapminder_sample("tiny")

print(get_html(df))
df

## Pandas: Inspecting and summarizing data

In [ ]:
# Example

df = get_gapminder_sample("full")
df = pd.concat([df.loc[:6], df.loc[1699:]])
df.loc[6] = "..."
new_index = df.index.tolist()
new_index[6] = "..."
df.index = new_index

print(get_html(df))
df

In [ ]:
# Summarize an entire DataFrame

df = get_gapminder_sample("full")

relevant = ["life_exp", "pop", "gdp_per_cap"]
to_print = df[relevant].describe()

print(get_html(to_print))
to_print

## Pandas: Functional Data Management

In [ ]:
# Example
from epp_topics.config import SRC

df = pd.read_csv(SRC / "pandas" / "functional" / "survey.csv")
print(get_html(df))
df

## Pandas: Data Types

In [ ]:
# The need for different data types

df = get_gapminder_sample("tiny")
print(get_html(df))
df

## Pandas: DataFrames and Series

In [ ]:
# What is a DataFrame

df = get_gapminder_sample("tiny")
print(get_html(df))
df

In [ ]:
# Creating DataFrames and Series

df = pd.DataFrame(
    data=[[1, "bla"], [3, "blubb"]],
    columns=["a", "b"],
    index=["c", "d"],
)
print(get_html(df))
df

In [ ]:
# Assigment is index aligned!

df = pd.DataFrame(
    data=[[1, "bla"], [3, "blubb"]],
    columns=["a", "b"],
    index=["c", "d"],
)

sr = pd.Series(
    [2.71, 3.14],
    index=["d", "c"],
)
df["new_col"] = sr

print(get_html(df))
df

## Pandas: Creating Variables

In [ ]:
# Using numpy math functions

import numpy as np

df = get_gapminder_sample("tiny")
df["log_life_exp"] = np.log(df["life_exp"])

print(get_html(df))
df

In [ ]:
# Arithmetic with Series

df = get_gapminder_sample("full")
df = df.query("country in ('Cuba', 'Spain')")
df = df.query("year in (2002, 2007)")
df = df[["country", "year", "gdp_per_cap", "pop"]]
df = df.reset_index(drop=True)
df
df["gdp_billion"] = df["gdp_per_cap"] * df["pop"] / 1e9

print(get_html(df))
df

In [ ]:
# Recoding values

df = get_gapminder_sample("tiny")
df["country_code"] = df["country"].replace(
    {"Cuba": "CUB", "Spain": "ESP"},
)

print(get_html(df))
df

In [ ]:
# Vectorized if conditions

df = get_gapminder_sample("full")
df = df.query("country in ('Cuba', 'Spain')")
df = df.query("year in (2002, 2007)")
df = df[["country", "year", "gdp_per_cap", "pop"]]
df = df.reset_index(drop=True)

helper = pd.Series(
    "rich",
    index=df.index,
)

df["income_status"] = helper.where(
    cond=df["gdp_per_cap"] > 10000,
    other="not rich",
)

print(get_html(df))
df

## Pandas: Selecting rows and columns

In [ ]:
# Selecting columns

df = get_gapminder_sample("tiny")
df = df[["country", "continent"]]

print(get_html(df))
df

In [ ]:
# Selecting individual rows

df = get_gapminder_sample("tiny")
df = df.set_index(["country", "year"])
to_print = df.loc[[1, 3], ["country", "year"]]

print(get_html(to_print))
to_print

In [ ]:
# Selecting rows and columns

df = get_gapminder_sample("tiny")
to_print = df.loc[[1, 3], ["country", "year"]]

print(get_html(to_print))
to_print

In [ ]:
# Selecting rows using Boolean Series

df = get_gapminder_sample("tiny")
to_print = df[df["year"] >= 2005]

print(get_html(to_print))
to_print

In [ ]:
# Selecting rows with queries (1)

df = get_gapminder_sample("tiny")
to_print = df.query("year >= 2005")

print(get_html(to_print))
to_print

In [ ]:
# Selecting rows with queries (2)

df = get_gapminder_sample("tiny")
to_print = df.query("year >= 2005 & continent == 'Europe'")

print(get_html(to_print))
to_print

## Software engineering: Testing code that should raise errors

In [ ]:
# Reminder of the example

from epp_topics.config import SRC

df = pd.read_csv(SRC / "pandas" / "functional" / "survey.csv")
print(get_html(df))
df

## Software engineering: What does pytest do?

In [ ]:
# Example

from epp_topics.config import SRC

df = pd.read_csv(SRC / "pandas" / "functional" / "survey.csv")
print(get_html(df))
df